In this notebook we showcase the usage of FedCG (A secure federalized kernel least square learning algorithm) as shown in Algorithm 1 of the paper. 

First we explore the datasets that we used for our tests (Table 1 from the paper), then we show how to use the class FedCG, then run the simulations described in Section 3 (Experimental results). 



In [1]:
# Import necessary libraries and modules
import numpy as np
import pdb
from sklearn.metrics import accuracy_score


# importing scripts for auxiliary functions
import sys
import os
sys.path.append('..')
sys.path.append('./py_scripts')


from py_scripts.kernel import Kernel
from py_scripts.dataset_handler import *
from py_scripts.federated_functions import FedCG 
from py_scripts.simulation import model_simulation, nys_simulation

In [38]:
# INITIALISATION
# Define your data, parameters, and model initialization
# kernel type can be: 0 for linear, 1 for RBF, 2 for polynomial, 3 for random kernel, 4 wu kernel
# sigma: width/variance of the rbf kernel, default is None
# polydegree: degree of the polynomial kernel, default is None
# binary: 1 for yes, default is None


kernel_params = Kernel(kernel_type=1, kernel_sigma = 1, poly_degree = None, binary = None) 
nyst_points = 50    # Define the number of Nystrom points
lam = 1e-6  # Define the regularization parameter - è a 1e-6 per tutti tranne sonar e ionosphere
Nb = 10             # Define the number of batches or hospitals
toll = 1e-3      # Define the tolerance value - a 1e-3 per tutti meno sonar e ionosphere

# Explore datasets


### Toy dataset

In [ ]:
[X_train, y_train, X_test, y_test, W ]=load_random_gen_dataset( N=2000, feat=3, seed=0, nystr_pt=nyst_points, nystr_method='subsampling')

visualize_dataset_pca_with_W(X_train, y_train, W, 'TRAIN generated, W subsampling')
visualize_dataset_pca(X_test, y_test,'TEST generated')

### Iris

In [ ]:
[X_train, y_train, X_test, y_test, W ]= load_iris_dataset(seed=0, nystr_pt=50, nystr_method='subsampling')

visualize_dataset_pca_with_W(X_train, y_train, W, 'TRAIN Iris, W subsampling')
visualize_dataset_pca(X_test, y_test,'TEST Iris')

### Ionosphere

In [ ]:
[X_train, y_train, X_test, y_test, W ]= load_ionosphere_dataset(seed=0, nystr_pt=50, nystr_method='subsampling')

visualize_dataset_pca_with_W(X_train, y_train, W, 'TRAIN Ionosphere subsampling')
visualize_dataset_pca(X_test, y_test,'TEST Ionosphere')

### Sonar

In [ ]:
[X_train, y_train, X_test, y_test, W ]= load_sonar_dataset(seed=0, nystr_pt=50, nystr_method='subsampling')

visualize_dataset_pca_with_W(X_train, y_train, W, 'TRAINING Sonar subsampling')
visualize_dataset_pca(X_test, y_test,'TEST Sonar')

### BC Wisconsin

In [ ]:
[X_train, y_train, X_test, y_test, W ]= load_bc_dataset(seed=0, nystr_pt=50, nystr_method='subsampling')

visualize_dataset_pca_with_W(X_train, y_train, W, 'TRAIN BC Wisconsin subsampling')
visualize_dataset_pca(X_test, y_test,'TEST BCWisconsin normal')

### Wine

In [ ]:
[X_train, y_train, X_test, y_test, W ]= load_wine_dataset(seed=0, nystr_pt=50, nystr_method='subsampling')

visualize_dataset_pca_with_W(X_train, y_train, W, 'TRAIN Wine subsampling')
visualize_dataset_pca(X_test, y_test,'TEST Wine')

# Using the class - Toy dataset

In [ ]:
[X_train, y_train, X_test, y_test, W ]=load_random_gen_dataset( N=2000, feat=3, seed=0, nystr_pt=nyst_points, nystr_method='uniform')
#visualize_dataset_pca_with_W(X_train, y_train, W, "Generated_ds_W_uniform")

# Initialise the FedCG object
fedcg_model = FedCG(kernel_params, nyst_points, lam, Nb, toll)
# We also initialise a CenCG centralised model for comparison
cencg_model = FedCG(kernel_params, nyst_points, lam, 1, toll)

#TRAINING
localResidue, federated_accuracy, test_accuracy, elapsed, alpha = fedcg_model.fit(X_train, y_train, X_test, y_test, W=W, alpha_init=None)


In [ ]:
central_localResidue, central_federated_accuracy, cen_time, central_test_accuracy , central_alpha = cencg_model.fit(X_train, y_train, X_test, y_test, W=W, alpha_init=None)

In [ ]:
#PREDICTIONS
predictions = fedcg_model.predict(X_test)
transformed_predictions = np.sign(predictions)
print("Unique values in predictions: ", np.unique(transformed_predictions))  # Display unique values in transformed_predictions

# Compute accuracy
accuracy = accuracy_score(y_test, transformed_predictions)
print(f"Accuracy of the federated model on the test set: {accuracy}")

In [ ]:
# CENTRALISED PREDICTIONS
cen_predictions = cencg_model.predict(X_test)
cen_transformed_predictions = np.sign(cen_predictions)

# Compute accuracy
cen_accuracy = accuracy_score(y_test, cen_transformed_predictions)
print(f"Accuracy of the centralised model on the test set: {cen_accuracy}")

## Testing with actual datasets

### Iris dataset

In [ ]:
[X_iris_train, y_iris_train, X_iris_test, y_iris_test, W_iris ]=load_iris_dataset(nystr_pt=nyst_points, nystr_method='uniform')

# Initialise the FedCG object
fedcg_model_iris = FedCG(kernel_params, nyst_points, lam, Nb, toll)
# We also initialise a CenCG centralised model for comparison
cencg_model_iris = FedCG(kernel_params, nyst_points, lam, 1, toll)

iris_localResidue, iris_federated_accuracy, iris_test_accuracy,exp_time,  iris_alpha = fedcg_model_iris.fit(X_iris_train, y_iris_train, X_iris_test, y_iris_test, W=W_iris, alpha_init=None)


In [ ]:
central_iris_localResidue, cent_iris_federated_accuracy, cent_iris_test_accuracy, cen_ext_time,  central_iris_alpha = cencg_model_iris.fit(X_iris_train, y_iris_train, X_iris_test, y_iris_test, W=W_iris, alpha_init=None)

### Ionosphere dataset

In [ ]:
[X_ionosphere_train, y_ionosphere_train, X_ionosphere_test, y_ionosphere_test, W_ionosphere ]=load_ionosphere_dataset(nystr_pt=nyst_points, nystr_method='uniform')

In [ ]:
# Define the range of values for lambda and toll for grid search
lam_values = [1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10, 1e-11]  # Add more values if needed
toll_values = [1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2]  # Add more values if needed

best_epoch = 0
best_params = {}

for lam in lam_values:
    for toll in toll_values:
        fedcg_model_ionosphere = FedCG(kernel_params, nyst_points, lam, Nb, toll)
        ionosphere_localResidue, ionosphere_federated_accuracy, ionosphere_test_accuracy, ionosphere_elapsed, ionosphere_alpha = fedcg_model_ionosphere.fit(X_ionosphere_train, y_ionosphere_train, X_ionosphere_test, y_ionosphere_test, W=W_ionosphere, alpha_init=None, silent=True)
        stopping_epoch = len(ionosphere_localResidue)
        if stopping_epoch > best_epoch:
            best_epoch = stopping_epoch
            best_params['lambda'] = lam
            best_params['tolerance'] = toll
            best_model = fedcg_model_ionosphere

print("Best stopping epoch:", best_epoch)
print("Best lambda:", best_params['lambda'])
print("Best tolerance:", best_params['tolerance'])


In [ ]:
lam = 1e-6
toll =  1e-3

# Initialise the FedCG object
fedcg_model_ionosphere = FedCG(kernel_params, nyst_points, lam, Nb, toll)
# We also initialise a CenCG centralised model for comparison
cencg_model_ionosphere = FedCG(kernel_params, nyst_points, lam, 1, toll)

ionosphere_localResidue, ionosphere_federated_accuracy, ionosphere_test_accuracy, ionosphere_elapsed, ionosphere_alpha = fedcg_model_ionosphere.fit(X_ionosphere_train, y_ionosphere_train, X_ionosphere_test, y_ionosphere_test, W=W_ionosphere, alpha_init=None)

In [ ]:
#PREDICTIONS
predictions = fedcg_model_ionosphere.predict(X_ionosphere_test)
transformed_predictions = np.sign(predictions)
print("Unique values in predictions: ", np.unique(transformed_predictions))  # Display unique values in transformed_predictions

# Compute accuracy
accuracy = accuracy_score(y_ionosphere_test, transformed_predictions)
print(f"Accuracy of the federated model on the test set: {accuracy}")

In [ ]:
central_ionosphere_localResidue, cent_ionosphere_federated_accuracy, cent_ionosphere_test_accuracy,central_ionosphere_elapsed, central_ionosphere_alpha = cencg_model_ionosphere.fit(X_ionosphere_train, y_ionosphere_train, X_ionosphere_test, y_ionosphere_test, W=W_ionosphere, alpha_init=None)

### Sonar dataset

In [ ]:
[X_sonar_train, y_sonar_train, X_sonar_test, y_sonar_test, W_sonar ]=load_sonar_dataset(nystr_pt=nyst_points, nystr_method='uniform')


In [ ]:
# Define the range of values for lambda and toll for grid search
lam_values = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10, 1e-11]  # Add more values if needed
toll_values = [1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]  # Add more values if needed

best_epoch = 0
best_params = {}

for lam in lam_values:
    for toll in toll_values:
        fedcg_model_sonar = FedCG(kernel_params, nyst_points, lam, Nb, toll)
        sonar_localResidue, sonar_federated_accuracy, sonar_test_accuracy, sonar_elapsed, sonar_alpha = fedcg_model_sonar.fit(X_sonar_train, y_sonar_train, X_sonar_test, y_sonar_test, W=W_sonar, alpha_init=None, silent=True)
        stopping_epoch = len(sonar_localResidue)
        if stopping_epoch > best_epoch:
            best_epoch = stopping_epoch
            best_params['lambda'] = lam
            best_params['tolerance'] = toll
            best_model = fedcg_model_sonar

print("Best stopping epoch:", best_epoch)
print("Best lambda:", best_params['lambda'])
print("Best tolerance:", best_params['tolerance'])


In [ ]:
lam = 1e-6
toll = 1e-3
fedcg_model_sonar = FedCG(kernel_params, nyst_points, lam, Nb, toll)
cencg_model_sonar = FedCG(kernel_params, nyst_points, lam, 1, toll)

sonar_localResidue, sonar_federated_accuracy, sonar_test_accuracy,sonar_elapsed, sonar_alpha = fedcg_model_sonar.fit(X_sonar_train, y_sonar_train, X_sonar_test, y_sonar_test, W=W_sonar, alpha_init=None)

In [ ]:
#PREDICTIONS
predictions = fedcg_model_sonar.predict(X_sonar_test)
transformed_predictions = np.sign(predictions)
print("Unique values in predictions: ", np.unique(transformed_predictions))  # Display unique values in transformed_predictions

# Compute accuracy
accuracy = accuracy_score(y_sonar_test, transformed_predictions)
print(f"Accuracy of the federated model on the test set: {accuracy}")

In [ ]:
central_sonar_localResidue, cen_sonar_federated_accuracy, cen_sonar_test_accuracy,central_sonar_elapsed, central_sonar_alpha = cencg_model_sonar.fit(X_sonar_train, y_sonar_train, X_sonar_test, y_sonar_test, W=W_sonar, alpha_init=None)

# Simulations
Random initialisation at each run, 10 runs, function `model_simulation` in `py_scripts/simulation.py`.

## Toy dataset

In [39]:
[X_train, y_train, X_test, y_test, W ]=load_random_gen_dataset( N=2000, feat=3, seed=0, nystr_pt=nyst_points, nystr_method='uniform')

model_simulation('Generated_uniform', 10, X_train, y_train, X_test, y_test, W, kernel_params, nyst_points, lam, Nb, toll)

Loading random generated dataset...
W matrix shape:  (50, 3)


## Iris dataset


In [40]:
[X_train, y_train, X_test, y_test, W ]=load_iris_dataset( nystr_pt=nyst_points, nystr_method='uniform')
model_simulation('Iris_uniform', 10, X_train, y_train, X_test, y_test, W, kernel_params, nyst_points, lam, Nb, toll)

Loading Iris dataset...
Features are: sepal length (cm), sepal width (cm), petal length (cm), petal width (cm)
Classes are: setosa, versicolor, virginica
but we'll consider 'setosa' as one class and 'versicolor'+'virginica' as another class
y_train and y_test shape: (105,) (45,)
X_train and X_test shape: (105, 4) (45, 4)
W matrix shape:  (50, 4)


## Ionosphere dataset

In [41]:
lam = 1e-4
toll =  1e-6
[X_train, y_train, X_test, y_test, W ]=load_ionosphere_dataset(nystr_pt=nyst_points, nystr_method='uniform')
model_simulation('Ionosphere_uniform', 10, X_train, y_train, X_test, y_test, W, kernel_params, nyst_points, lam, Nb, toll)

## Sonar dataset

In [42]:
[X_train, y_train, X_test, y_test, W ]=load_sonar_dataset( nystr_pt=nyst_points, nystr_method='uniform')
lam = 1e-6
toll = 1e-6
model_simulation('Sonar_uniform', 10, X_train, y_train, X_test, y_test, W, kernel_params, nyst_points, lam, Nb, toll)

Loading Sonar dataset...
Unique labels in the dataset: ['Mine' 'Rock']


## Breast cancer

In [43]:
lam = 1e-6
toll = 1e-3
[X_train, y_train, X_test, y_test, W ]=load_bc_dataset( nystr_pt=nyst_points, nystr_method='uniform')
model_simulation('Breast_cancer_uniform', 10, X_train, y_train, X_test, y_test, W, kernel_params, nyst_points, lam, Nb, toll)

Loading the Breast Cancer Wisconsin dataset...
X_train and X_test shape: (398, 30) (171, 30)
y_train and y_test shape: (398,) (171,)


## Wine dataset

In [44]:
[X_train, y_train, X_test, y_test, W ]=load_wine_dataset( nystr_pt=nyst_points, nystr_method='uniform')
model_simulation('Wine_uniform', 10, X_train, y_train, X_test, y_test, W, kernel_params, nyst_points, lam, Nb, toll)

Loading Wine Recognition dataset...
Features are: alcohol, malic_acid, ash, alcalinity_of_ash, magnesium, total_phenols, flavanoids, nonflavanoid_phenols, proanthocyanins, color_intensity, hue, od280/od315_of_diluted_wines, proline
Classes are: class_0, class_1, class_2
We will consider 'class_1' as one class and 'class_0'+'class_2' as another
y_train and y_test shape: (124,) (54,)
X_train and X_test shape: (124, 13) (54, 13)
W matrix shape:  (50, 13)


# Nystrom matrix simulations

Nystrom landmark new initialisation at each run, 10 runs, function `nys_simulation` in `py_scripts/simulation.py`.

In [ ]:
[X_train, y_train, X_test, y_test]=load_random_gen_dataset(N=2000, feat=3, seed=0, nystr_pt=nyst_points, nystr_method='uniform')[:-1]
nys_simulation('Generated', 10, X_train, y_train, X_test, y_test, 'uniform', kernel_params, nyst_points, lam, Nb, toll)


In [ ]:
[X_train, y_train, X_test, y_test]=load_iris_dataset( nystr_pt=nyst_points, nystr_method='uniform')[:-1]
nys_simulation('Iris', 10, X_train, y_train, X_test, y_test, 'uniform', kernel_params, nyst_points, lam, Nb, toll)

In [ ]:
lam = 1e-4
toll =  1e-6
[X_train, y_train, X_test, y_test ]=load_ionosphere_dataset(nystr_pt=nyst_points, nystr_method='uniform')[:-1]
nys_simulation('Ionosphere', 10, X_train, y_train, X_test, y_test, 'uniform', kernel_params, nyst_points, lam, Nb, toll)

In [ ]:
lam = 1e-6
toll =  1e-4
[X_train, y_train, X_test, y_test ]=load_sonar_dataset( nystr_pt=nyst_points, nystr_method='uniform')[:-1]
nys_simulation('Sonar', 10, X_train, y_train, X_test, y_test, 'uniform', kernel_params, nyst_points, lam, Nb, toll)

In [ ]:
[X_train, y_train, X_test, y_test ]=load_bc_dataset( nystr_pt=nyst_points, nystr_method='uniform')[:-1]
nys_simulation('BreastCancer', 10, X_train, y_train, X_test, y_test, 'uniform', kernel_params, nyst_points, lam, Nb, toll)

In [ ]:
[X_train, y_train, X_test, y_test ]=load_wine_dataset( nystr_pt=nyst_points, nystr_method='uniform')[:-1]   
nys_simulation('Wine', 10, X_train, y_train, X_test, y_test, 'uniform', kernel_params, nyst_points, lam, Nb, toll)

# Summary tables


## FedCG vs CenCG, random initialisation
Fixed Nystrom landmarks matrix.
Average results for 10 simulations

### uniform

In [45]:

fedCG_generated_uniform = "Results_FedCG/Random_Initialisation/fedCG/Generated/fedcg_results_Generated_uniform.csv"  # Replace with the actual file path on your machine
get_stats_latex(fedCG_generated_uniform)

${16.80 \pm 0.84}$ & ${4.03 \pm 1.50}$ & ${0.94 \pm 0.00}$ & ${0.96 \pm 0.00}$ & ${0.95 \pm 0.00}$


In [46]:
cenCG_generated_uniform = "Results_FedCG/Random_Initialisation/cenCG/Generated/cencg_results_Generated_uniform.csv"
get_stats_latex(cenCG_generated_uniform)

${16.70 \pm 0.97}$ & ${4.63 \pm 3.30}$ & ${0.94 \pm 0.00}$ & ${0.96 \pm 0.00}$ & ${0.95 \pm 0.00}$


In [47]:
fedCG_Iris_uniform = "Results_FedCG/Random_Initialisation/fedCG/Iris/fedcg_results_Iris_uniform.csv"
get_stats_latex(fedCG_Iris_uniform)

${9.80 \pm 1.58}$ & ${0.21 \pm 0.06}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$


In [48]:
cenGC_Iris_uniform = "Random_Initialisation/cenCG/Iris/cencg_results_Iris_uniform.csv"
get_stats_latex(cenGC_Iris_uniform)

${10.10 \pm 1.99}$ & ${0.05 \pm 0.04}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$


In [49]:
fedCG_Ionosphere_uniform = "Results_FedCG/Random_Initialisation/fedCG/Ionosphere/fedcg_results_Ionosphere_uniform.csv"
get_stats_latex(fedCG_Ionosphere_uniform)

${60.00 \pm 4.52}$ & ${12.08 \pm 2.30}$ & ${0.89 \pm 0.00}$ & ${1.00 \pm 0.00}$ & ${0.84 \pm 0.00}$


In [50]:
cenCG_Ionosphere_uniform = "Results_FedCG/Random_Initialisation/cenCG/Ionosphere/cencg_results_Ionosphere_uniform.csv"
get_stats_latex(cenCG_Ionosphere_uniform)

${60.80 \pm 4.30}$ & ${2.52 \pm 0.87}$ & ${0.89 \pm 0.00}$ & ${1.00 \pm 0.00}$ & ${0.84 \pm 0.00}$


In [51]:
fedCG_Sonar_uniform = "Results_FedCG/Random_Initialisation/fedCG/Sonar/fedcg_results_Sonar_uniform.csv"
get_stats_latex(fedCG_Sonar_uniform)

${48.90 \pm 0.63}$ & ${15.90 \pm 3.72}$ & ${0.90 \pm 0.00}$ & ${0.86 \pm 0.00}$ & ${0.86 \pm 0.00}$


In [52]:
cenCG_Sonar_uniform = "Results_FedCG/Random_Initialisation/cenCG/Sonar/cencg_results_Sonar_uniform.csv"
get_stats_latex(cenCG_Sonar_uniform)

${49.10 \pm 0.63}$ & ${2.56 \pm 0.74}$ & ${0.90 \pm 0.00}$ & ${0.86 \pm 0.00}$ & ${0.95 \pm 0.00}$


In [53]:
fedCG_BreastCancer_uniform = "Results_FedCG/Random_Initialisation/fedCG/BreastCancer/fedcg_results_Breast_cancer_uniform.csv"
get_stats_latex(fedCG_BreastCancer_uniform)

${42.30 \pm 1.35}$ & ${7.15 \pm 1.74}$ & ${0.94 \pm 0.00}$ & ${0.84 \pm 0.00}$ & ${0.98 \pm 0.00}$


In [54]:
cenCG_BreastCancer_uniform = "Results_FedCG/Random_Initialisation/cenCG/BreastCancer/cencg_results_Breast_cancer_uniform.csv"
get_stats_latex(cenCG_BreastCancer_uniform)

${42.20 \pm 3.86}$ & ${1.54 \pm 0.43}$ & ${0.94 \pm 0.00}$ & ${0.84 \pm 0.00}$ & ${0.98 \pm 0.00}$


In [55]:
fedCG_Wine_uniform = "Results_FedCG/Random_Initialisation/fedCG/Wine/fedcg_results_Wine_uniform.csv"
get_stats_latex(fedCG_Wine_uniform)

${37.80 \pm 2.07}$ & ${2.78 \pm 0.83}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$


In [56]:
cenCG_Wine_uniform = "Results_FedCG/Random_Initialisation/cenCG/Wine/cencg_results_Wine_uniform.csv"
get_stats_latex(cenCG_Wine_uniform)

${37.10 \pm 2.57}$ & ${0.35 \pm 0.15}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$


### normal

In [57]:

fedCG_generated_normal = "Results_FedCG/Random_Initialisation/fedCG/Generated/fedcg_results_Generated_normal.csv"  # Replace with the actual file path
get_stats_latex(fedCG_generated_normal)

${13.00 \pm 0.00}$ & ${7.74 \pm 3.27}$ & ${0.94 \pm 0.00}$ & ${0.96 \pm 0.00}$ & ${0.95 \pm 0.00}$


In [58]:
cenCG_generated_normal= "Results_FedCG/Random_Initialisation/cenCG/Generated/cencg_results_Generated_normal.csv"
get_stats_latex(cenCG_generated_normal)

${13.00 \pm 0.00}$ & ${6.43 \pm 2.95}$ & ${0.94 \pm 0.00}$ & ${0.96 \pm 0.00}$ & ${0.95 \pm 0.00}$


In [59]:
fedCG_Iris_normal = "Results_FedCG/Random_Initialisation/fedCG/Iris/fedcg_results_Iris_normal.csv"
get_stats_latex(fedCG_Iris_normal)

${11.20 \pm 0.84}$ & ${0.31 \pm 0.20}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$


In [60]:
cenGC_Iris_normal = "Results_FedCG/Random_Initialisation/cenCG/Iris/cencg_results_Iris_normal.csv"
get_stats_latex(cenGC_Iris_normal)

${11.10 \pm 1.75}$ & ${0.08 \pm 0.06}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$


In [61]:
fedCG_Ionosphere_normal = "Results_FedCG/Random_Initialisation/fedCG/Ionosphere/fedcg_results_Ionosphere_normal.csv"
get_stats_latex(fedCG_Ionosphere_normal)

${66.30 \pm 3.89}$ & ${8.67 \pm 2.90}$ & ${0.85 \pm 0.00}$ & ${0.95 \pm 0.00}$ & ${0.84 \pm 0.00}$


In [62]:
cenCG_Ionosphere_normal = "Results_FedCG/Random_Initialisation/cenCG/Ionosphere/cencg_results_Ionosphere_normal.csv"
get_stats_latex(cenCG_Ionosphere_normal)

${67.00 \pm 3.77}$ & ${4.87 \pm 2.66}$ & ${0.85 \pm 0.00}$ & ${0.95 \pm 0.00}$ & ${0.82 \pm 0.00}$


In [63]:
fedCG_Sonar_normal = "Results_FedCG/Random_Initialisation/fedCG/Sonar/fedcg_results_Sonar_normal.csv"
get_stats_latex(fedCG_Sonar_normal)

${47.30 \pm 3.66}$ & ${7.51 \pm 1.34}$ & ${0.83 \pm 0.00}$ & ${0.85 \pm 0.00}$ & ${0.79 \pm 0.00}$


In [64]:
cenCG_Sonar_normal = "Results_FedCG/Random_Initialisation/cenCG/Sonar/cencg_results_Sonar_normal.csv"
get_stats_latex(cenCG_Sonar_normal)

${48.40 \pm 2.35}$ & ${3.07 \pm 1.18}$ & ${0.83 \pm 0.00}$ & ${0.85 \pm 0.00}$ & ${0.77 \pm 0.00}$


In [65]:
fedCG_BreastCancer_normal = "Results_FedCG/Random_Initialisation/fedCG/BreastCancer/fedcg_results_Breast_cancer_normal.csv"
get_stats_latex(fedCG_BreastCancer_normal)

${55.80 \pm 4.09}$ & ${4.82 \pm 0.85}$ & ${0.95 \pm 0.00}$ & ${0.89 \pm 0.00}$ & ${0.98 \pm 0.00}$


In [66]:
cenCG_BreastCancer_normal = "Results_FedCG/Random_Initialisation/cenCG/BreastCancer/cencg_results_Breast_cancer_normal.csv"
get_stats_latex(cenCG_BreastCancer_normal)

${53.70 \pm 7.00}$ & ${2.64 \pm 0.87}$ & ${0.95 \pm 0.00}$ & ${0.89 \pm 0.00}$ & ${0.98 \pm 0.01}$


In [67]:
fedCG_Wine_normal = "Results_FedCG/Random_Initialisation/fedCG/Wine/fedcg_results_Wine_normal.csv"
get_stats_latex(fedCG_Wine_normal)

${40.70 \pm 3.53}$ & ${1.48 \pm 0.17}$ & ${0.96 \pm 0.00}$ & ${0.91 \pm 0.00}$ & ${1.00 \pm 0.00}$


In [68]:
cenCG_Wine_normal = "Results_FedCG/Random_Initialisation/cenCG/Wine/cencg_results_Wine_normal.csv"
get_stats_latex(cenCG_Wine_normal)

${41.00 \pm 3.77}$ & ${0.28 \pm 0.03}$ & ${0.96 \pm 0.00}$ & ${0.91 \pm 0.00}$ & ${1.00 \pm 0.00}$


### subsampling

In [10]:

fedCG_generated_subsampling = "Results_FedCG/Random_Initialisation/fedCG/Generated/fedcg_results_Generated_subsampling.csv"  # Replace with the actual file path
get_stats_latex(fedCG_generated_subsampling)

${11.90 \pm 0.63}$ & ${3.29 \pm 1.88}$ & ${0.94 \pm 0.00}$ & ${0.96 \pm 0.00}$ & ${0.95 \pm 0.00}$


In [11]:
cenCG_generated_subsampling= "Results_FedCG/Random_Initialisation/cenCG/Generated/cencg_results_Generated_subsampling.csv"
get_stats_latex(cenCG_generated_subsampling)

${11.90 \pm 0.63}$ & ${3.97 \pm 2.89}$ & ${0.94 \pm 0.00}$ & ${0.96 \pm 0.00}$ & ${0.95 \pm 0.00}$


In [12]:
fedCG_Iris_subsampling = "Results_FedCG/Random_Initialisation/fedCG/Iris/fedcg_results_Iris_subsampling.csv"
get_stats_latex(fedCG_Iris_subsampling)

${9.00 \pm 0.00}$ & ${0.23 \pm 0.13}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$


In [13]:
cenGC_Iris_subsampling = "Results_FedCG/Random_Initialisation/cenCG/Iris/cencg_results_Iris_subsampling.csv"
get_stats_latex(cenGC_Iris_subsampling)

${8.90 \pm 0.63}$ & ${0.04 \pm 0.04}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$


In [14]:
fedCG_Ionosphere_subsampling = "Results_FedCG/Random_Initialisation/fedCG/Ionosphere/fedcg_results_Ionosphere_subsampling.csv"
get_stats_latex(fedCG_Ionosphere_subsampling)

${139.30 \pm 2.99}$ & ${17.29 \pm 3.52}$ & ${0.92 \pm 0.00}$ & ${0.95 \pm 0.00}$ & ${0.88 \pm 0.00}$


In [15]:
cenCG_Ionosphere_subsampling = "Results_FedCG/Random_Initialisation/cenCG/Ionosphere/cencg_results_Ionosphere_subsampling.csv"
get_stats_latex(cenCG_Ionosphere_subsampling)

${140.40 \pm 2.53}$ & ${5.19 \pm 3.11}$ & ${0.92 \pm 0.00}$ & ${0.95 \pm 0.00}$ & ${0.89 \pm 0.00}$


In [16]:
fedCG_Sonar_subsampling = "Results_FedCG/Random_Initialisation/fedCG/Sonar/fedcg_results_Sonar_subsampling.csv"
get_stats_latex(fedCG_Sonar_subsampling)

${46.80 \pm 1.26}$ & ${9.05 \pm 1.80}$ & ${0.89 \pm 0.00}$ & ${0.78 \pm 0.00}$ & ${0.91 \pm 0.00}$


In [17]:
cenCG_Sonar_subsampling = "Results_FedCG/Random_Initialisation/cenCG/Sonar/cencg_results_Sonar_subsampling.csv"
get_stats_latex(cenCG_Sonar_subsampling)

${42.40 \pm 3.55}$ & ${1.76 \pm 0.33}$ & ${0.89 \pm 0.00}$ & ${0.78 \pm 0.00}$ & ${0.95 \pm 0.00}$


In [18]:
fedCG_BreastCancer_subsampling = "Results_FedCG/Random_Initialisation/fedCG/BreastCancer/fedcg_results_Breast_cancer_subsampling.csv"
get_stats_latex(fedCG_BreastCancer_subsampling)

${77.20 \pm 7.35}$ & ${8.63 \pm 2.92}$ & ${0.96 \pm 0.00}$ & ${0.95 \pm 0.00}$ & ${0.95 \pm 0.00}$


In [19]:
cenCG_BreastCancer_subsampling = "Results_FedCG/Random_Initialisation/cenCG/BreastCancer/cencg_results_Breast_cancer_subsampling.csv"
get_stats_latex(cenCG_BreastCancer_subsampling)

${78.80 \pm 13.16}$ & ${2.70 \pm 1.34}$ & ${0.96 \pm 0.00}$ & ${0.95 \pm 0.00}$ & ${0.95 \pm 0.00}$


In [20]:
fedCG_Wine_subsampling = "Results_FedCG/Random_Initialisation/fedCG/Wine/fedcg_results_Wine_subsampling.csv"
get_stats_latex(fedCG_Wine_subsampling)

${48.40 \pm 4.73}$ & ${2.12 \pm 0.41}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$


In [21]:
cenCG_Wine_subsampling = "Results_FedCG/Random_Initialisation/cenCG/Wine/cencg_results_Wine_subsampling.csv"
get_stats_latex(cenCG_Wine_subsampling)

${47.40 \pm 6.27}$ & ${0.37 \pm 0.12}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$


## Federated vs Central, changing Nystrom matrix
Fixed initialisation. Average results for 10 simulations

### uniform

In [81]:

nyst_fedCG_generated_uniform = "Results_FedCG/Random_Nystrom_Matrix/NystFedCG/Generated/nystr_fedcg_results_Generated_uniform.csv"
get_stats_latex(nyst_fedCG_generated_uniform)

${18.60 \pm 5.51}$ & ${6.29 \pm 4.44}$ & ${0.94 \pm 0.00}$ & ${0.96 \pm 0.00}$ & ${0.95 \pm 0.00}$


In [82]:
nyst_cenCG_generated_uniform = "Results_FedCG/Random_Nystrom_Matrix/NystCenCG/Generated/nystr_cencg_results_Generated_uniform.csv"
get_stats_latex(nyst_cenCG_generated_uniform)

${18.30 \pm 5.50}$ & ${4.86 \pm 2.59}$ & ${0.94 \pm 0.00}$ & ${0.96 \pm 0.00}$ & ${0.95 \pm 0.00}$


In [83]:
nyst_fedCG_iris_uniform = "Results_FedCG/Random_Nystrom_Matrix/NystFedCG/Iris/nystr_fedcg_results_Iris_uniform.csv"
get_stats_latex(nyst_fedCG_iris_uniform)

${17.80 \pm 5.15}$ & ${0.37 \pm 0.13}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$


In [84]:
nyst_cenCG_iris_uniform = "Results_FedCG/Random_Nystrom_Matrix/NystCenCG/Iris/nystr_cencg_results_Iris_uniform.csv"
get_stats_latex(nyst_cenCG_iris_uniform)

${18.40 \pm 4.24}$ & ${0.09 \pm 0.06}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$


In [85]:
nyst_fedCG_ionosphere_uniform = "Results_FedCG/Random_Nystrom_Matrix/NystFedCG/Ionosphere/nystr_fedcg_results_Ionosphere_uniform.csv"
get_stats_latex(nyst_fedCG_ionosphere_uniform)

${61.00 \pm 9.19}$ & ${6.30 \pm 1.54}$ & ${0.83 \pm 0.03}$ & ${0.99 \pm 0.05}$ & ${0.79 \pm 0.03}$


In [86]:
nyst_cenCG_ionosphere_uniform = "Results_FedCG/Random_Nystrom_Matrix/NystCenCG/Ionosphere/nystr_cencg_results_Ionosphere_uniform.csv"
get_stats_latex(nyst_cenCG_ionosphere_uniform)

${61.40 \pm 7.32}$ & ${2.31 \pm 1.02}$ & ${0.83 \pm 0.03}$ & ${0.99 \pm 0.04}$ & ${0.78 \pm 0.03}$


In [87]:
nyst_fedCG_sonar_uniform = "Results_FedCG/Random_Nystrom_Matrix/NystFedCG/Sonar/nystr_fedcg_results_Sonar_uniform.csv"
get_stats_latex(nyst_fedCG_sonar_uniform)

${27.30 \pm 5.74}$ & ${4.64 \pm 1.25}$ & ${0.71 \pm 0.07}$ & ${0.74 \pm 0.15}$ & ${0.62 \pm 0.10}$


In [88]:
nyst_cenCG_sonar_uniform = "Results_FedCG/Random_Nystrom_Matrix/NystCenCG/Sonar/nystr_cencg_results_Sonar_uniform.csv"
get_stats_latex(nyst_cenCG_sonar_uniform)

${26.20 \pm 5.23}$ & ${1.33 \pm 0.93}$ & ${0.71 \pm 0.07}$ & ${0.72 \pm 0.11}$ & ${0.64 \pm 0.07}$


In [89]:
nyst_fedCG_BreastCancer_uniform = "Results_FedCG/Random_Nystrom_Matrix/NystFedCG/BreastCancer/nystr_fedcg_results_BreastCancer_uniform.csv"
get_stats_latex(nyst_fedCG_BreastCancer_uniform)

${59.80 \pm 15.34}$ & ${5.42 \pm 1.32}$ & ${0.94 \pm 0.03}$ & ${0.87 \pm 0.07}$ & ${0.96 \pm 0.04}$


In [90]:
nyst_cenCG_BreastCancer_uniform = "Results_FedCG/Random_Nystrom_Matrix/NystCenCG/BreastCancer/nystr_cencg_results_BreastCancer_uniform.csv"
get_stats_latex(nyst_cenCG_BreastCancer_uniform)

${59.00 \pm 16.22}$ & ${2.42 \pm 1.60}$ & ${0.94 \pm 0.03}$ & ${0.86 \pm 0.09}$ & ${0.97 \pm 0.04}$


In [91]:
nyst_fedCG_Wine_uniform = "Results_FedCG/Random_Nystrom_Matrix/NystFedCG/Wine/nystr_fedcg_results_Wine_uniform.csv"
get_stats_latex(nyst_fedCG_Wine_uniform)

${64.60 \pm 12.90}$ & ${3.48 \pm 2.02}$ & ${0.96 \pm 0.05}$ & ${0.90 \pm 0.12}$ & ${0.99 \pm 0.03}$


In [92]:
nyst_cenCG_Wine_uniform = "Results_FedCG/Random_Nystrom_Matrix/NystCenCG/Wine/nystr_cencg_results_Wine_uniform.csv"
get_stats_latex(nyst_cenCG_Wine_uniform)

${63.50 \pm 13.41}$ & ${0.53 \pm 0.30}$ & ${0.96 \pm 0.05}$ & ${0.92 \pm 0.11}$ & ${1.00 \pm 0.00}$


### normal

In [93]:

nyst_fedCG_generated_normal = "Results_FedCG/Random_Nystrom_Matrix/NystFedCG/Generated/nystr_fedcg_results_Generated_normal.csv"
get_stats_latex(nyst_fedCG_generated_normal)

${12.60 \pm 1.93}$ & ${4.36 \pm 1.95}$ & ${0.94 \pm 0.00}$ & ${0.97 \pm 0.01}$ & ${0.95 \pm 0.00}$


In [94]:
nyst_cenCG_generated_normal = "Results_FedCG/Random_Nystrom_Matrix/NystCenCG/Generated/nystr_cencg_results_Generated_normal.csv"
get_stats_latex(nyst_cenCG_generated_normal)

${13.30 \pm 4.12}$ & ${3.76 \pm 1.47}$ & ${0.94 \pm 0.00}$ & ${0.96 \pm 0.01}$ & ${0.95 \pm 0.00}$


In [95]:
nyst_fedCG_iris_normal= "Results_FedCG/Random_Nystrom_Matrix/NystFedCG/Iris/nystr_fedcg_results_Iris_normal.csv"
get_stats_latex(nyst_fedCG_iris_normal)

${10.00 \pm 2.83}$ & ${0.20 \pm 0.07}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$


In [96]:
nyst_cenCG_iris_normal = "Results_FedCG/Random_Nystrom_Matrix/NystCenCG/Iris/nystr_cencg_results_Iris_normal.csv"
get_stats_latex(nyst_cenCG_iris_normal)

${10.50 \pm 2.71}$ & ${0.04 \pm 0.02}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$


In [97]:
nyst_fedCG_ionosphere_normal = "Results_FedCG/Random_Nystrom_Matrix/NystFedCG/Ionosphere/nystr_fedcg_results_Ionosphere_normal.csv"
get_stats_latex(nyst_fedCG_ionosphere_normal)

${64.40 \pm 7.00}$ & ${6.99 \pm 1.92}$ & ${0.87 \pm 0.03}$ & ${0.99 \pm 0.02}$ & ${0.82 \pm 0.05}$


In [98]:
nyst_cenCG_ionosphere_normal = "Results_FedCG/Random_Nystrom_Matrix/NystCenCG/Ionosphere/nystr_cencg_results_Ionosphere_normal.csv"
get_stats_latex(nyst_cenCG_ionosphere_normal)

${63.90 \pm 6.29}$ & ${2.60 \pm 1.44}$ & ${0.87 \pm 0.03}$ & ${0.99 \pm 0.02}$ & ${0.83 \pm 0.04}$


In [99]:
nyst_fedCG_sonar_normal = "Results_FedCG/Random_Nystrom_Matrix/NystFedCG/Sonar/nystr_fedcg_results_Sonar_normal.csv"
get_stats_latex(nyst_fedCG_sonar_normal)

${35.90 \pm 5.12}$ & ${6.18 \pm 1.55}$ & ${0.77 \pm 0.10}$ & ${0.68 \pm 0.16}$ & ${0.74 \pm 0.19}$


In [100]:
nyst_cenCG_sonar_normal = "Results_FedCG/Random_Nystrom_Matrix/NystCenCG/Sonar/nystr_cencg_results_Sonar_normal.csv"
get_stats_latex(nyst_cenCG_sonar_normal)

${36.80 \pm 5.48}$ & ${1.89 \pm 1.02}$ & ${0.77 \pm 0.10}$ & ${0.67 \pm 0.13}$ & ${0.77 \pm 0.18}$


In [101]:
nyst_fedCG_BreastCancer_normal = "Results_FedCG/Random_Nystrom_Matrix/NystFedCG/BreastCancer/nystr_fedcg_results_BreastCancer_normal.csv"
get_stats_latex(nyst_fedCG_BreastCancer_normal)

${74.60 \pm 12.04}$ & ${6.84 \pm 0.97}$ & ${0.96 \pm 0.02}$ & ${0.92 \pm 0.04}$ & ${0.97 \pm 0.03}$


In [102]:
nyst_cenCG_BreastCancer_normal = "Results_FedCG/Random_Nystrom_Matrix/NystCenCG/BreastCancer/nystr_cencg_results_BreastCancer_normal.csv"
get_stats_latex(nyst_cenCG_BreastCancer_normal)

${76.00 \pm 11.31}$ & ${3.12 \pm 1.19}$ & ${0.96 \pm 0.02}$ & ${0.91 \pm 0.03}$ & ${0.98 \pm 0.04}$


In [103]:
nyst_fedCG_Wine_normal = "Results_FedCG/Random_Nystrom_Matrix/NystFedCG/Wine/nystr_fedcg_results_Wine_normal.csv"
get_stats_latex(nyst_fedCG_Wine_normal)

${66.00 \pm 16.97}$ & ${2.68 \pm 0.52}$ & ${0.97 \pm 0.04}$ & ${0.92 \pm 0.14}$ & ${1.00 \pm 0.00}$


In [104]:
nyst_cenCG_Wine_normal = "Results_FedCG/Random_Nystrom_Matrix/NystCenCG/Wine/nystr_cencg_results_Wine_normal.csv"
get_stats_latex(nyst_cenCG_Wine_normal)

${65.40 \pm 15.12}$ & ${0.42 \pm 0.08}$ & ${0.97 \pm 0.04}$ & ${0.93 \pm 0.12}$ & ${1.00 \pm 0.00}$


### subsampling

In [69]:

nyst_fedCG_generated_subsampling = "Results_FedCG/Random_Nystrom_Matrix/NystFedCG/Generated/nystr_fedcg_results_Generated_subsampling.csv"
get_stats_latex(nyst_fedCG_generated_subsampling)

${12.40 \pm 1.40}$ & ${3.56 \pm 2.88}$ & ${0.94 \pm 0.00}$ & ${0.96 \pm 0.01}$ & ${0.95 \pm 0.00}$


In [70]:
nyst_cenCG_generated_subsampling = "Results_FedCG/Random_Nystrom_Matrix/NystCenCG/Generated/nystr_cencg_results_Generated_subsampling.csv"
get_stats_latex(nyst_cenCG_generated_subsampling)

${12.30 \pm 0.97}$ & ${3.16 \pm 2.75}$ & ${0.94 \pm 0.00}$ & ${0.96 \pm 0.01}$ & ${0.95 \pm 0.00}$


In [71]:
nyst_fedCG_iris_subsampling= "Results_FedCG/Random_Nystrom_Matrix/NystFedCG/Iris/nystr_fedcg_results_Iris_subsampling.csv"
get_stats_latex(nyst_fedCG_iris_subsampling)

${9.00 \pm 0.00}$ & ${0.21 \pm 0.06}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$


In [72]:
nyst_cenCG_iris_subsampling = "Results_FedCG/Random_Nystrom_Matrix/NystCenCG/Iris/nystr_cencg_results_Iris_subsampling.csv"
get_stats_latex(nyst_cenCG_iris_subsampling)

${9.00 \pm 0.00}$ & ${0.04 \pm 0.02}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$ & ${1.00 \pm 0.00}$


In [73]:
nyst_fedCG_ionosphere_subsampling = "Results_FedCG/Random_Nystrom_Matrix/NystFedCG/Ionosphere/nystr_fedcg_results_Ionosphere_subsampling.csv"
get_stats_latex(nyst_fedCG_ionosphere_subsampling)

${136.30 \pm 23.89}$ & ${13.27 \pm 2.77}$ & ${0.89 \pm 0.04}$ & ${0.95 \pm 0.04}$ & ${0.87 \pm 0.06}$


In [74]:
nyst_cenCG_ionosphere_subsampling = "Results_FedCG/Random_Nystrom_Matrix/NystCenCG/Ionosphere/nystr_cencg_results_Ionosphere_subsampling.csv"
get_stats_latex(nyst_cenCG_ionosphere_subsampling)

${140.70 \pm 29.14}$ & ${5.18 \pm 1.38}$ & ${0.89 \pm 0.04}$ & ${0.96 \pm 0.03}$ & ${0.87 \pm 0.06}$


In [75]:
nyst_fedCG_sonar_subsampling = "Results_FedCG/Random_Nystrom_Matrix/NystFedCG/Sonar/nystr_fedcg_results_Sonar_subsampling.csv"
get_stats_latex(nyst_fedCG_sonar_subsampling)

${52.90 \pm 16.53}$ & ${8.76 \pm 2.91}$ & ${0.86 \pm 0.07}$ & ${0.83 \pm 0.15}$ & ${0.86 \pm 0.07}$


In [76]:
nyst_cenCG_sonar_subsampling = "Results_FedCG/Random_Nystrom_Matrix/NystCenCG/Sonar/nystr_cencg_results_Sonar_subsampling.csv"
get_stats_latex(nyst_cenCG_sonar_subsampling)

${53.30 \pm 18.38}$ & ${2.81 \pm 1.13}$ & ${0.86 \pm 0.07}$ & ${0.81 \pm 0.13}$ & ${0.87 \pm 0.06}$


In [77]:
nyst_fedCG_BreastCancer_subsampling = "Results_FedCG/Random_Nystrom_Matrix/NystFedCG/BreastCancer/nystr_fedcg_results_BreastCancer_subsampling.csv"
get_stats_latex(nyst_fedCG_BreastCancer_subsampling)

${102.90 \pm 23.50}$ & ${8.82 \pm 2.50}$ & ${0.97 \pm 0.02}$ & ${0.94 \pm 0.04}$ & ${0.97 \pm 0.05}$


In [78]:
nyst_cenCG_BreastCancer_subsampling = "Results_FedCG/Random_Nystrom_Matrix/NystCenCG/BreastCancer/nystr_cencg_results_BreastCancer_subsampling.csv"
get_stats_latex(nyst_cenCG_BreastCancer_subsampling)

${109.10 \pm 26.25}$ & ${4.02 \pm 1.33}$ & ${0.97 \pm 0.02}$ & ${0.94 \pm 0.03}$ & ${0.98 \pm 0.03}$


In [79]:
nyst_fedCG_Wine_subsampling = "Results_FedCG/Random_Nystrom_Matrix/NystFedCG/Wine/nystr_fedcg_results_Wine_subsampling.csv"
get_stats_latex(nyst_fedCG_Wine_subsampling)

${77.60 \pm 21.06}$ & ${3.40 \pm 1.01}$ & ${0.98 \pm 0.03}$ & ${0.94 \pm 0.07}$ & ${1.00 \pm 0.00}$


In [80]:
nyst_cenCG_Wine_subsampling = "Results_FedCG/Random_Nystrom_Matrix/NystCenCG/Wine/nystr_cencg_results_Wine_subsampling.csv"
get_stats_latex(nyst_cenCG_Wine_subsampling)

${80.30 \pm 18.67}$ & ${0.53 \pm 0.14}$ & ${0.98 \pm 0.03}$ & ${0.95 \pm 0.08}$ & ${1.00 \pm 0.03}$
